![aga](img/AB_logo.png)

# Indexes and Index-Organized Tables

Indexes are schema objects that can speed access to table rows. Index-organized tables are tables stored in an index structure.

## Introduction to Indexes

An **index** is an optional structure, associated with a table or **table cluster**, that can sometimes speed data access. Indexes are schema objects that are logically and physically independent of the data in the objects with which they are associated. Thus, you can drop or create an index without physically affecting the indexed table.

For an analogy, suppose an HR manager has a shelf of cardboard boxes. Folders containing employee information are inserted randomly in the boxes. The folder for employee Whalen (ID 200) is 10 folders up from the bottom of box 1, whereas the folder for King (ID 100) is at the bottom of box 3. To locate a folder, the manager looks at every folder in box 1 from bottom to top, and then moves from box to box until the folder is found. To speed access, the manager could create an index that sequentially lists every employee ID with its folder location:

```
ID 100: Box 3, position 1 (bottom)
ID 101: Box 7, position 8 
ID 200: Box 1, position 10
.
.
.
```

Similarly, the manager could create separate indexes for employee last names, department IDs, and so on.

## Benefits of Indexes

The absence or presence of an index does not require a change in the wording of any SQL statement. An index is a fast access path to a single row of data. It affects only the speed of execution. Given a data value that has been indexed, the index points directly to the location of the rows containing that value.

When an index exists on one or more columns of a table, the database can in some cases retrieve a small set of randomly distributed rows from the table. Indexes are one of many means of reducing disk I/O. If a heap-organized table has no indexes, then the database must perform a **[full table scan](https://docs.oracle.com/database/121/CNCPT/glossary.htm#GUID-BF9B54D6-892E-4C3B-8536-38958ACC069D)** to find a value. For example, a **[query](https://docs.oracle.com/database/121/CNCPT/glossary.htm#GUID-CCF91C9F-A98A-498F-A84B-58A0FA16CD6E)** of location `2700` in the unindexed `hr.departments` table requires the database to search every row in every block. This approach does not scale well as data volumes increase.

In general, consider creating an index on a column in any of the following situations:

* The indexed columns are queried frequently and return a small percentage of the total number of rows in the table.


* A referential **[integrity constraint](https://docs.oracle.com/database/121/CNCPT/glossary.htm#GUID-67F8FE8C-EBA5-4796-820A-8919982A1411)** exists on the indexed column or columns. The index is a means to avoid a full table **[lock](https://docs.oracle.com/database/121/CNCPT/glossary.htm#GUID-6D016291-A487-4F88-BE0B-ACF8FA2AE72C)** that would otherwise be required if you update the parent table **[primary key](https://docs.oracle.com/database/121/CNCPT/glossary.htm#GUID-8640EFA5-276C-4812-A078-1F21F55F4200)**, merge into the parent table, or delete from the parent table.


* A unique key constraint will be placed on the table and you want to manually specify the index and all index options.

## Index Usability and Visibility

Indexes are usable (default) or unusable, visible (default) or invisible.

These properties are defined as follows:

* Usability

  An **[unusable index](https://docs.oracle.com/database/121/CNCPT/glossary.htm#GUID-5EEB4F35-818F-4478-8BE3-F70CF22CD11F)**, which is ignored by the **[optimizer](https://docs.oracle.com/database/121/CNCPT/glossary.htm#GUID-54114749-0A81-41D7-8E16-7B76D93CEE2B)**, is not maintained by DML operations. An unusable index can improve the performance of bulk loads. Instead of dropping an index and later re-creating it, you can make the index unusable and then rebuild it. Unusable indexes and index partitions do not consume space. When you make a usable index unusable, the database drops its index **[segment](https://docs.oracle.com/database/121/CNCPT/glossary.htm#GUID-EC12AA68-8C89-43B3-B1F9-3AABF7CAEB9F)**.


* Visibility

  An **[invisible index](https://docs.oracle.com/database/121/CNCPT/glossary.htm#GUID-B60609DA-2397-4715-B7E2-75AEC3FAD0BF)** is maintained by DML operations, but is not used by default by the optimizer. Making an index invisible is an alternative to making it unusable or dropping it. Invisible indexes are especially useful for testing the removal of an index before dropping it or using indexes temporarily without affecting the overall application.

## Keys and Columns

A **key** is a set of columns or expressions on which you can build an index. Although the terms are often used interchangeably, indexes and keys are different. Indexes are structures stored in the database that users manage using SQL statements. Keys are strictly a logical concept.

The following statement creates an index on the `customer_id` column of the sample table `oe.orders`:

`CREATE INDEX ord_customer_ix ON orders (customer_id);`

In the preceding statement, the `customer_id` column is the index key. The index itself is named `ord_customer_ix`.

## Composite Indexes

A **composite index**, also called a **concatenated index**, is an index on multiple columns in a table. Place columns in a composite index in the order that makes the most sense for the queries that will retrieve data. The columns need not be adjacent in the table.

Composite indexes can speed retrieval of data for `SELECT` statements in which the `WHERE` clause references all or the leading portion of the columns in the composite index. Therefore, the order of the columns used in the definition is important. In general, the most commonly accessed columns go first.

For example, suppose an application frequently queries the `last_name`, `job_id`, and `salary` columns in the `employees` table. Also assume that `last_name` has high **[cardinality](https://docs.oracle.com/database/121/CNCPT/glossary.htm#GUID-5CD22620-6D7A-40DC-BA09-EE3B5339C7F8)**, which means that the number of distinct values is large compared to the number of table rows. You create an index with the following column order:

```
CREATE INDEX employees_ix
   ON employees (last_name, job_id, salary);
```

Queries that access all three columns, only the `last_name` column, or only the `last_name` and `job_id` columns use this index. In this example, queries that do not access the `last_name` column do not use the index.

Multiple indexes can exist on the same table with the same column order when they meet any of the following conditions:

* The indexes are of different types.

  For example, you can create bitmap and B-tree indexes on the same columns.


* The indexes use different partitioning schemes.

  For example, you can create indexes that are locally partitioned and indexes that are globally partitioned.


* The indexes have different uniqueness properties.

  For example, you can create both a unique and a non-unique index on the same set of columns.

For example, a nonpartitioned index, global partitioned index, and locally partitioned index can exist for the same table columns in the same order. Only one index with the same number of columns in the same order can be visible at any one time.

This capability enables you to migrate applications without the need to drop an existing index and re-create it with different attributes. Also, this capability is useful in an OLTP database when an index key keeps increasing, causing the database to insert new entries into the same set of index blocks. To alleviate such "hot spots," you could evolve the index from a nonpartitioned index into a global partitioned index.

If indexes on the same set of columns do not differ in type or partitioning scheme, then these indexes must use different column permutations. For example, the following SQL statements specify valid column permutations:

```
CREATE INDEX employee_idx1 ON employees (last_name, job_id);
CREATE INDEX employee_idx2 ON employees (job_id, last_name);
```

## Unique and Nonunique Indexes

Indexes can be unique or nonunique. Unique indexes guarantee that no two rows of a table have duplicate values in the key column or columns.

For example, your application may require that no two employees have the same employee ID. In a unique index, one rowid exists for each data value. The data in the leaf blocks is sorted only by key.

Nonunique indexes permit duplicates values in the indexed column or columns. For example, the `first_name` column of the `employees` table may contain multiple `Mike` values. For a nonunique index, the rowid is included in the key in sorted order, so nonunique indexes are sorted by the index key and rowid (ascending).

Oracle Database does not index table rows in which all key columns are **[null](https://docs.oracle.com/database/121/CNCPT/glossary.htm#GUID-8854502F-2B8F-4ABC-98FA-BBFC3695A964)**, except for bitmap indexes or when the cluster key column value is null.

## Types of Indexes

Oracle Database provides several indexing schemes, which provide complementary performance functionality.

Indexes are categorized as follows:

* B-tree indexes

  These indexes are the standard index type. They are excellent for highly selective indexes (few rows correspond to each index entry) and primary key indexes. Used as concatenated indexes, a **[B-tree index](https://docs.oracle.com/database/121/CNCPT/glossary.htm#GUID-8D6D0C64-6AC8-4B22-A9AF-1B62F61AE10B)** can retrieve data sorted by the indexed columns. B-tree indexes have the following subtypes:  

   * Index-organized tables

     An index-organized table differs from a heap-organized because the data is itself the index. See "[Overview of Index-Organized Tables](https://docs.oracle.com/database/121/CNCPT/indexiot.htm#GUID-DAEC075B-C16D-4A57-898C-70EBCB364F0C)**".

   * Reverse key indexes

     In this type of index, the bytes of the index key are reversed, for example, 103 is stored as 301. The reversal of bytes spreads out inserts into the index over many blocks. See "[Reverse Key Indexes](https://docs.oracle.com/database/121/CNCPT/indexiot.htm#GUID-2646BDA9-F776-4C98-9487-C7EBC2EECF0B)".

   * Descending indexes

     This type of index stores data on a particular column or columns in descending order. See "[Ascending and Descending Indexes](https://docs.oracle.com/database/121/CNCPT/indexiot.htm#GUID-8C2EA2EC-18E5-4E4A-BF74-D1DE86D7F24A)".

   * B-tree cluster indexes

     This type indexes a table cluster key. Instead of pointing to a row, the key points to the block that contains rows related to the cluster key. See "[Overview of Indexed Clusters](https://docs.oracle.com/database/121/CNCPT/tablecls.htm#GUID-CC31365B-83B0-4E09-A047-BF1B79AC887A)".


* Bitmap and bitmap join indexes

  In a bitmap index, an index entry uses a bitmap to point to multiple rows. In contrast, a B-tree index entry points to a single row. A bitmap join index is a bitmap index for the join of two or more tables. See "[Overview of Bitmap Indexes](https://docs.oracle.com/database/121/CNCPT/indexiot.htm#GUID-B15C4817-7748-456D-9740-8B9628AF9F47)".


* Function-based indexes

  This type of index includes columns that are either transformed by a function, such as the `UPPER` function, or included in an expression. B-tree or bitmap indexes can be function-based. See "[Overview of Function-Based Indexes](https://docs.oracle.com/database/121/CNCPT/indexiot.htm#GUID-9AD7651D-0F0D-4FC6-A984-5845F0224EE6)".


* Application domain indexes

  A user creates this type of index for data in an application-specific domain. The physical index need not use a traditional index structure and can be stored either in the Oracle database as tables or externally as a file. See "[Overview of Application Domain Indexes](https://docs.oracle.com/database/121/CNCPT/indexiot.htm#GUID-9586EB86-4B84-4A43-A66D-958776FE558B)".


## How the Database Maintains Indexes
The database automatically maintains and uses indexes after they are created. Indexes automatically reflect data changes, such as adding, updating, and deleting rows in their underlying tables, with no additional actions required by users.

Retrieval performance of indexed data remains almost constant, even as rows are inserted. However, the presence of many indexes on a table degrades **[DML](https://docs.oracle.com/database/121/CNCPT/glossary.htm#GUID-B5F2F112-1B33-41B5-B63D-9DC8F99A369D)** performance because the database must also update the indexes.

### Index Storage
Oracle Database stores index data in an index segment. Space available for index data in a data block is the data block size minus block overhead, entry overhead, rowid, and one length byte for each value indexed.

The **[tablespace](https://docs.oracle.com/database/121/CNCPT/glossary.htm#GUID-AA66891C-71B2-4D55-8F64-0E427AE24E88)** of an index segment is either the default tablespace of the owner or a tablespace specifically named in the `CREATE INDEX` statement. For ease of administration you can store an index in a separate tablespace from its table. For example, you may choose not to back up tablespaces containing only indexes, which can be rebuilt, and so decrease the time and storage required for backups.

# Overview of B-Tree Indexes

B-trees, short for balanced trees, are the most common type of database index. A **B-tree index** is an ordered list of values divided into ranges. By associating a key with a row or range of rows, B-trees provide excellent retrieval performance for a wide range of queries, including exact match and range searches.

The following figure illustrates the structure of a B-tree index. The example shows an index on the `department_id` column, which is a foreign key column in the `employees` table.

***Figure 3-1 Internal Structure of a B-tree Index***
![Figure 3-1](https://docs.oracle.com/database/121/CNCPT/img/GUID-83032A7C-64B8-4705-A1A4-9289C791DBE2-default.gif)

## Branch Blocks and Leaf Blocks

A B-tree index has two types of blocks: the **branch block** for searching, and the **leaf block** for storing key values. The upper-level branch blocks of a B-tree index contain index data that points to lower-level index blocks.

In Figure 3-1, the root branch block has an entry `0-40`, which points to the leftmost block in the next branch level. This branch block contains entries such as `0-10` and `11-19`. Each of these entries points to a leaf block that contains key values that fall in the range.

A B-tree index is balanced because all leaf blocks automatically stay at the same depth. Thus, retrieval of any record from anywhere in the index takes approximately the same amount of time. The height of the index is the number of blocks required to go from the root block to a leaf block. The branch level is the height minus 1. In Figure 3-1, the index has a height of 3 and a branch level of 2.

Branch blocks store the minimum key prefix needed to make a branching decision between two keys. This technique enables the database to fit as much data as possible on each branch block. The branch blocks contain a pointer to the child block containing the key. The number of keys and pointers is limited by the block size.

The leaf blocks contain every indexed data value and a corresponding rowid used to locate the actual row. Each entry is sorted by (key, rowid). Within a leaf block, a key and rowid is linked to its left and right sibling entries. The leaf blocks themselves are also doubly linked. In Figure 3-1 the leftmost leaf block (`0-10`) is linked to the second leaf block (`11-19`).

## Index Scans

In an **index scan**, the database retrieves a row by traversing the index, using the indexed column values specified by the statement. If the database scans the index for a value, then it will find this value in *n* I/Os where *n* is the height of the B-tree index. This is the basic principle behind Oracle Database indexes.

If a SQL statement accesses only indexed columns, then the database reads values directly from the index rather than from the table. If the statement accesses nonindexed columns in addition to the indexed columns, then the database uses rowids to find the rows in the table. Typically, the database retrieves table data by alternately reading an index block and then a table block.

### Full Index Scan

In a **full index scan**, the database reads the entire index in order. A full index scan is available if a **predicate** (`WHERE` clause) in the SQL statement references a column in the index, and in some circumstances when no predicate is specified. A full scan can eliminate sorting because the data is ordered by index key.

***Example 3-1 Full Index Scan***

Suppose that an application runs the following query:

```
SELECT department_id, last_name, salary 
FROM   employees
WHERE  salary > 5000 
ORDER BY department_id, last_name;
```

In this example, the `department_id`, `last_name`, and `salary` are a composite key in an index. Oracle Database performs a full scan of the index, reading it in sorted order (ordered by department ID and last name) and filtering on the salary attribute. In this way, the database scans a set of data smaller than the `employees` table, which contains more columns than are included in the query, and avoids sorting the data.

The full scan could read the index entries as follows:

```
50,Atkinson,2800,rowid
60,Austin,4800,rowid
70,Baer,10000,rowid
80,Abel,11000,rowid
80,Ande,6400,rowid
110,Austin,7200,rowid
.
.
.
```

### Fast Full Index Scan

A **fast full index scan** is a full index scan in which the database accesses the data in the index itself without accessing the table, and the database reads the index blocks in no particular order.

Fast full index scans are an alternative to a **[full table scan](https://docs.oracle.com/database/121/CNCPT/glossary.htm#GUID-BF9B54D6-892E-4C3B-8536-38958ACC069D)** when both of the following conditions are met:

* The index must contain all columns needed for the query.


* A row containing all nulls must not appear in the query result set. For this result to be guaranteed, at least one column in the index must have either:

  * A `NOT NULL` constraint


  * A predicate applied to the column that prevents nulls from being considered in the query result set


***Example 3-2 Fast Full Index Scan***

Assume that an application issues the following query, which does not include an `ORDER BY` clause:

```
SELECT last_name, salary
FROM   employees;
```

The `last_name` column has a not null constraint. If the last name and salary are a composite key in an index, then a fast full index scan can read the index entries to obtain the requested information:

```
Baida,2900,rowid
Atkinson,2800,rowid
Zlotkey,10500,rowid
Austin,7200,rowid
Baer,10000,rowid
Austin,4800,rowid
.
.
.
```

### Index Range Scan
An **index range scan** is an ordered scan of an index in which one or more leading columns of an index are specified in conditions, and 0, 1, or more values are possible for an index key.

A **[condition](https://docs.oracle.com/database/121/CNCPT/glossary.htm#GUID-B5AA8627-E7DC-487B-8D4B-2DE3F1497A83)** specifies a combination of one or more expressions and logical (Boolean) operators and returns a value of `TRUE`, `FALSE`, or `UNKNOWN`.

The database commonly uses an index range scan to access selective data. The **[selectivity](https://docs.oracle.com/database/121/CNCPT/glossary.htm#GUID-AA830B4F-E5E8-4CCC-A960-0FA0E2F8DE12)** is the percentage of rows in the table that the query selects, with 0 meaning no rows and 1 meaning all rows. Selectivity is tied to a query **[predicate](https://docs.oracle.com/database/121/CNCPT/glossary.htm#GUID-891CF9E9-78CD-470C-9C4A-D65A101B2C38)**, such as `WHERE last_name LIKE 'A%'`, or a combination of predicates. A predicate becomes more selective as the value approaches 0 and less selective (or more unselective) as the value approaches 1.

For example, a user queries employees whose last names begin with `A`. Assume that the `last_name` column is indexed, with entries as follows:

```
Abel,rowid
Ande,rowid
Atkinson,rowid
Austin,rowid
Austin,rowid
Baer,rowid
.
.
.
```

The database could use a range scan because the `last_name` column is specified in the predicate and multiples rowids are possible for each index key. For example, two employees are named Austin, so two rowids are associated with the key `Austin`.

An index range scan can be bounded on both sides, as in a query for departments with IDs between 10 and 40, or bounded on only one side, as in a query for IDs over 40. To scan the index, the database moves backward or forward through the leaf blocks. For example, a scan for IDs between 10 and 40 locates the first index leaf block that contains the lowest key value that is 10 or greater. The scan then proceeds horizontally through the linked list of leaf nodes until it locates a value greater than 40.

### Index Unique Scan

In contrast to an index range scan, an **index unique scan** must have either 0 or 1 rowid associated with an index key. The database performs a unique scan when a predicate references all of the columns in the key of a `UNIQUE` index using an equality operator. An index unique scan stops processing as soon as it finds the first record because no second record is possible.

As an illustration, suppose that a user runs the following query:

```
SELECT *
FROM   employees
WHERE  employee_id = 5;
```

Assume that the `employee_id` column is the primary key and is indexed with entries as follows:

```
1,rowid
2,rowid
4,rowid
5,rowid
6,rowid
.
.
.
```

In this case, the database can use an index unique scan to locate the rowid for the employee whose ID is 5.


### Index Skip Scan

An **index skip scan** uses logical subindexes of a composite index. The database "skips" through a single index as if it were searching separate indexes.

Skip scanning is beneficial if there are few distinct values in the leading column of a composite index and many distinct values in the nonleading key of the index. The database may choose an index skip scan when the leading column of the composite index is not specified in a query predicate.

***Example 3-3 Skip Scan of a Composite Index***

Assume that you run the following query for a customer in the `sh.customers` table:

`SELECT * FROM sh.customers WHERE cust_email = 'Abbey@company.example.com';`

The `customers` table has a column `cust_gender` whose values are either `M` or `F`. Assume that a composite index exists on the columns (`cust_gender`, `cust_email`). The following example shows a portion of the index entries:

```
F,Wolf@company.example.com,rowid
F,Wolsey@company.example.com,rowid
F,Wood@company.example.com,rowid
F,Woodman@company.example.com,rowid
F,Yang@company.example.com,rowid
F,Zimmerman@company.example.com,rowid
M,Abbassi@company.example.com,rowid
M,Abbey@company.example.com,rowid
```

The database can use a skip scan of this index even though `cust_gender` is not specified in the `WHERE` clause.

In a skip scan, the number of logical subindexes is determined by the number of distinct values in the leading column. In the preceding example, the leading column has two possible values. The database logically splits the index into one subindex with the key `F` and a second subindex with the key `M`.

When searching for the record for the customer whose email is `Abbey@company.example.com`, the database searches the subindex with the value `F` first and then searches the subindex with the value `M`. Conceptually, the database processes the query as follows:

```
SELECT * FROM sh.customers WHERE cust_gender = 'F' 
  AND cust_email = 'Abbey@company.example.com'
UNION ALL
SELECT * FROM sh.customers WHERE cust_gender = 'M'
  AND cust_email = 'Abbey@company.example.com';
```

### Index Clustering Factor

The **index clustering factor** measures row order in relation to an indexed value such as employee last name. As the degree of order increases, the clustering factor decreases.

The clustering factor is useful as a rough measure of the number of I/Os required to read an entire table using an index:

* If the clustering factor is high, then Oracle Database performs a relatively high number of I/Os during a large index range scan. The index entries point to random table blocks, so the database may have to read and reread the same blocks over and over again to retrieve the data pointed to by the index.


* If the clustering factor is low, then Oracle Database performs a relatively low number of I/Os during a large index range scan. The index keys in a range tend to point to the same data block, so the database does not have to read and reread the same blocks over and over.


The clustering factor is relevant for index scans because it can show:

* Whether the database will use an index for large range scans


* The degree of table organization in relation to the index key


* Whether you should consider using an index-organized table, partitioning, or table cluster if rows must be ordered by the index key

***Example 3-4 Clustering Factor***

Assume that the `employees` table fits into two data blocks. Table 3-1 depicts the rows in the two data blocks (the ellipses indicate data that is not shown).

***Table 3-1 Contents of Two Data Blocks in the Employees Table***

| <b>Data Block 1</b>                   |
|:-----------------------------------|
| 100 Steven    <b>King</b>    SKING    ... |
| 156 Janette   <b>King</b>    JKING    ... |
| 115 Alexander <b>Khoo</b>    AKHOO    ... |
| . |
| . |
| . |
| 116 Shelli  <b>Baida</b>     SBAIDA   ... |
| 204 Hermann <b>Baer</b>      HBAER    ... |
| 105 David   <b>Austin</b>    DAUSTIN  ... |
| 130 Mozhe   <b>Atkinson</b>  MATKINSO ... |
| 166 Sundar  <b>Ande</b>      SANDE    ... |
| 174 Ellen   <b>Abel</b>      EABEL    ... |


| <b>Data Block 2</b>                  |
|:----------------------------------|
| 149 Eleni    <b>Zlotkey</b> EZLOTKEY ... |
| 200 Jennifer <b>Whalen</b>  JWHALEN  ... |
| . |
| . |
| . |
| 137 Renske   <b>Ladwig</b>  RLADWIG  ... |
| 173 Sundita  <b>Kumar</b>   SKUMAR   ... |
| 101 Neena    <b>Kochar</b>  NKOCHHAR ... |

Rows are stored in the blocks in order of last name (shown in bold). For example, the bottom row in data block 1 describes Abel, the next row up describes Ande, and so on alphabetically until the top row in block 1 for Steven King. The bottom row in block 2 describes Kochar, the next row up describes Kumar, and so on alphabetically until the last row in the block for Zlotkey.

Assume that an index exists on the last name column. Each name entry corresponds to a rowid. Conceptually, the index entries would look as follows:

```
Abel,block1row1
Ande,block1row2
Atkinson,block1row3
Austin,block1row4
Baer,block1row5
.
.
.
```

Assume that a separate index exists on the employee ID column. Conceptually, the index entries might look as follows, with employee IDs distributed in almost random locations throughout the two blocks:

```
100,block1row50
101,block2row1
102,block1row9
103,block2row19
104,block2row39
105,block1row4
.
.
.
```

The following statement queries the `ALL_INDEXES` view for the clustering factor for these two indexes:

```
SQL> SELECT INDEX_NAME, CLUSTERING_FACTOR 
  2  FROM ALL_INDEXES 
  3  WHERE INDEX_NAME IN ('EMP_NAME_IX','EMP_EMP_ID_PK');
 
INDEX_NAME           CLUSTERING_FACTOR
-------------------- -----------------
EMP_EMP_ID_PK                       19
EMP_NAME_IX                          2
```

The clustering factor for `EMP_NAME_IX` is low, which means that adjacent index entries in a single leaf block tend to point to rows in the same data blocks. The clustering factor for `EMP_EMP_ID_PK` is high, which means that adjacent index entries in the same leaf block are much less likely to point to rows in the same data blocks.

### Reverse Key Indexes

A **reverse key index** is a type of B-tree index that physically reverses the bytes of each index key while keeping the column order. For example, if the index key is `20`, and if the two bytes stored for this key in hexadecimal are `C1`,`15` in a standard B-tree index, then a reverse key index stores the bytes as `15`,`C1`.

Reversing the key solves the problem of contention for leaf blocks in the right side of a B-tree index. This problem can be especially acute in an Oracle Real Application Clusters (Oracle RAC) database in which multiple instances repeatedly modify the same block. For example, in an `orders` table the primary keys for orders are sequential. One instance in the cluster adds order 20, while another adds 21, with each instance writing its key to the same leaf block on the right-hand side of the index.

In a reverse key index, the reversal of the byte order distributes inserts across all leaf keys in the index. For example, keys such as 20 and 21 that would have been adjacent in a standard key index are now stored far apart in separate blocks. Thus, I/O for insertions of sequential keys is more evenly distributed.

Because the data in the index is not sorted by column key when it is stored, the reverse key arrangement eliminates the ability to run an index range scanning query in some cases. For example, if a user issues a query for order IDs greater than 20, then the database cannot start with the block containing this ID and proceed horizontally through the leaf blocks.

### Ascending and Descending Indexes
In an **ascending index**, Oracle Database stores data in ascending order. By default, character data is ordered by the binary values contained in each byte of the value, numeric data from smallest to largest number, and date from earliest to latest value.

For an example of an ascending index, consider the following SQL statement:

`CREATE INDEX emp_deptid_ix ON hr.employees(department_id);`

Oracle Database sorts the `hr.employees` table on the `department_id` column. It loads the ascending index with the `department_id` and corresponding rowid values in ascending order, starting with `0`. When it uses the index, Oracle Database searches the sorted `department_id` values and uses the associated rowids to locate rows having the requested `department_id` value.

By specifying the `DESC` keyword in the `CREATE INDEX` statement, you can create a **[descending index](https://docs.oracle.com/database/121/CNCPT/glossary.htm#GUID-77ADFA8C-BBA8-4E2D-B122-2D490FC35CFE)**. In this case, the index stores data on a specified column or columns in descending order. If the index in Table 3-1 on the `employees.department_id` column were descending, then the leaf blocking containing `250` would be on the left side of the tree and block with `0` on the right. The default search through a descending index is from highest to lowest value.

Descending indexes are useful when a query sorts some columns ascending and others descending. For an example, assume that you create a composite index on the `last_name` and `department_id` columns as follows:

`CREATE INDEX emp_name_dpt_ix ON hr.employees(last_name ASC, department_id DESC);`

If a user queries `hr.employees` for last names in ascending order (A to Z) and department IDs in descending order (high to low), then the database can use this index to retrieve the data and avoid the extra step of sorting it.

### Index Compression
To reduce space in indexes, Oracle Database can employ different compression algorithms.

#### Prefix Compression
Oracle Database can use **prefix compression**, also known as **key compression**, to compress portions of the primary key column values in a B-tree index or an index-organized table. Prefix compression can greatly reduce the space consumed by the index.

An uncompressed index entry has one piece. An index entry using prefix compression has two pieces: a prefix entry, which is the grouping piece, and a suffix entry, which is the unique or nearly unique piece. The database achieves compression by sharing the prefix entries among the suffix entries in an index block.

By default, the prefix of a unique index consists of all key columns excluding the last one, whereas the prefix of a nonunique index consists of all key columns. Suppose you create a composite, unique index on two columns of the `oe.orders` table as follows:

`CREATE UNIQUE INDEX orders_mod_stat_ix ON orders ( order_mode, order_status );`

In the preceding example, an index key might be `online`,`0`. The rowid is stored in the key data portion of the entry, and is not part of the key itself.

Alternatively, suppose you create a nonunique index on the same columns:

`CREATE INDEX orders_mod_stat_ix ON orders ( order_mode, order_status );`

Also assume that repeated values occur in the `order_mode` and `order_status` columns. An index block could have entries as shown in the follow example:

```
online,0,AAAPvCAAFAAAAFaAAa
online,0,AAAPvCAAFAAAAFaAAg
online,0,AAAPvCAAFAAAAFaAAl
online,2,AAAPvCAAFAAAAFaAAm
online,3,AAAPvCAAFAAAAFaAAq
online,3,AAAPvCAAFAAAAFaAAt
```

In the preceding example, the key prefix would consist of a concatenation of the `order_mode` and `order_status` values, as in `online`,`0`. The suffix consists in the rowid, as in `AAAPvCAAFAAAAFaAAa`. The rowid makes the whole index entry unique because a rowid is itself unique in the database.

If the index in the preceding example were created with default prefix compression (specified by the `COMPRESS` keyword), then duplicate key prefixes such as `online`,`0` and `online`,`3` would be compressed. Conceptually, the database achieves compression as follows:

```
online,0
AAAPvCAAFAAAAFaAAa
AAAPvCAAFAAAAFaAAg
AAAPvCAAFAAAAFaAAl
online,2
AAAPvCAAFAAAAFaAAm
online,3
AAAPvCAAFAAAAFaAAq
AAAPvCAAFAAAAFaAAt
```

Suffix entries (the rowids) form the compressed version of index rows. Each suffix entry references a prefix entry, which is stored in the same index block as the suffix.

Alternatively, you could specify a prefix length when creating an index that uses prefix compression. For example, if you specified `COMPRESS 1`, then the prefix would be `order_mode` and the suffix would be `order_status`,`rowid`. For the values in the index block example, the index would factor out duplicate occurrences of the prefix `online`, which can be represented conceptually as follows:

```
online
0,AAAPvCAAFAAAAFaAAa
0,AAAPvCAAFAAAAFaAAg
0,AAAPvCAAFAAAAFaAAl
2,AAAPvCAAFAAAAFaAAm
3,AAAPvCAAFAAAAFaAAq
3,AAAPvCAAFAAAAFaAAt
```

The index stores a specific prefix once per leaf block at most. Only keys in the leaf blocks of a B-tree index are compressed. In the branch blocks the key suffix can be truncated, but the key is not compressed.

### Advanced Index Compression

Starting with Oracle Database 12c Release 1 (12.1.0.2), **advanced index compression** improves on traditional prefix compression for indexes on heap-organized tables. Unlike prefix compression, which uses fixed duplicate key elimination for every block, advanced compression uses adaptive duplicate key elimination on a per-block basis.

Advanced index compression works at the block level in the following situations:

* During index creation, as a leaf block becomes full, the database automatically compresses the block to the optimal level.

* When reorganizing an index block as a result of DML, if the database can create sufficient space for the incoming index entry, then a block split does not occur. During DML without advanced index compression, however, an index block split always occurs when the block becomes full.

The main advantage of this form of compression is that the database automatically chooses the best compression for each block, so that the user does not require knowledge of data characteristics.

Enable advanced index compression using the `COMPRESS ADVANCED LOW` clause, as in the following example:

```
CREATE INDEX hr.emp_mndp_ix ON hr.employees(manager_id, department_id)
  COMPRESS ADVANCED LOW;
```